## Create TFRecords

In [70]:
import pandas as pd
import tensorflow as tf



bucket="/home/hx/Storage/Work/TUB/Datasets/WISENT-CIDDS-001/CIDDS-001/traffic/"
key="sanitized-cidds.tfrecord"

# Some initialization
pd.options.display.max_rows = 10

# Write column names at the start of our csv file
cols = ['Packets', 'Src IP Addr', 'Src Pt', 'Dst IP Addr', 'Dst Pt', 'Proto',
        'attackType']

csv_df = pd.DataFrame()

# Read each csv file and in the darkness bind them
for i in range(1, 5):
    df = pd.read_csv('{}CIDDS-001-external-week'.format(bucket) + str(i) + '.csv',
                            usecols=cols,chunksize=10).get_chunk()
    df['subnet'] = 1
    csv_df = csv_df.append(df)

for i in range(1, 5):
    df = pd.read_csv('{}CIDDS-001-internal-week'.format(bucket) + str(i) + '.csv',
                            usecols=cols, chunksize=10).get_chunk()
    df['subnet'] = 0
    csv_df = csv_df.append(df)
 

csv_df['attackType'] = csv_df['attackType'].map(lambda type: ['---','portScan','pingScan','dos','bruteForce'].index(type))

print(csv_df.info())


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
# example is a protocol buffer (under the hood)
example = tf.train.Example(features=tf.train.Features(feature={
   
    'src_ip_addr':_bytes_feature(m.encode('utf-8') for m in csv_df['Src IP Addr']),
    'src_pt': _int64_feature(csv_df['Src Pt'].values),
    'dst_ip_addr':_bytes_feature(m.encode('utf-8') for m in csv_df['Dst IP Addr']),
    'dst_pt': _int64_feature(csv_df['Dst Pt'].values),
    'proto': _bytes_feature(m.encode('utf-8') for m in csv_df['Proto']),
    'packets': _int64_feature(csv_df['Packets'].values,),
    'subnet': _int64_feature(csv_df['subnet'].values),
    'attackType': _int64_feature(csv_df['attackType'].values),
    
}))

processed_bucket='/home/hx/Storage/Work/TUB/Datasets/WISENT-CIDDS-001/CIDDS-001/'
processed_key = 'sanitized.tfrecord'
#s3.Object(processed_bucket, processed_key).put(Body=ex_string)

writer = tf.python_io.TFRecordWriter(processed_bucket+processed_key)
writer.write(example.SerializeToString())
writer.close()
print("Done")


print( tf.train.Example.FromString(example.SerializeToString()))

src_ip_size = csv_df['Src IP Addr'].count()
dst_ip_size = csv_df['Dst IP Addr'].count()
dst_pt_size = csv_df['Dst Pt'].count()
src_pt_size = csv_df['Src Pt'].count()
proto_size = csv_df['Proto'].count()
packets_size = csv_df['Packets'].count()
subnet_size = csv_df['subnet'].count()
attack_type_size = csv_df['attackType'].count()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 9
Data columns (total 8 columns):
Proto          80 non-null object
Src IP Addr    80 non-null object
Src Pt         80 non-null int64
Dst IP Addr    80 non-null object
Dst Pt         80 non-null int64
Packets        80 non-null int64
attackType     80 non-null int64
subnet         80 non-null int64
dtypes: int64(5), object(3)
memory usage: 5.6+ KB
None
Done
features {
  feature {
    key: "attackType"
    value {
      int64_list {
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
   

## Read TFRecords

In [80]:
  
def input_fn_train(repeat=10, batch_size=32):
    """
    Reads dataset from tfrecord, apply parser with map
    """
    # Import MNIST data
    dataset = tf.data.TFRecordDataset([processed_bucket+processed_key])

    # Map the parser over dataset, and batch results by up to batch_size
    dataset = dataset.map(_decode)
    dataset = dataset.repeat(repeat)
    return dataset

def _decode(serialized_ex):
    features={
 
        'src_ip_addr': tf.FixedLenFeature(src_ip_size,tf.string),
        'src_pt': tf.FixedLenFeature(src_pt_size,tf.int64),
        'dst_ip_addr': tf.FixedLenFeature(dst_ip_size,tf.string),
        'dst_pt': tf.FixedLenFeature(dst_pt_size,tf.int64),
        'proto': tf.FixedLenFeature(proto_size,tf.string),
        'packets': tf.FixedLenFeature(packets_size,tf.int64),
        'subnet': tf.FixedLenFeature(subnet_size,tf.int64),
        'attackType': tf.FixedLenFeature(attack_type_size,tf.int64)

    }
    parsed_features = tf.parse_single_example(serialized_ex, features)
    label = parsed_features.pop('attackType')
    return parsed_features, label


sess = tf.Session()

it = input_fn_train().make_one_shot_iterator()
print(sess.run(it.get_next()))  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])


({'dst_ip_addr': array([b'OPENSTACK_NET', b'EXT_SERVER', b'OPENSTACK_NET', b'EXT_SERVER',
       b'OPENSTACK_NET', b'EXT_SERVER', b'OPENSTACK_NET', b'EXT_SERVER',
       b'OPENSTACK_NET', b'EXT_SERVER', b'EXT_SERVER', b'19394_40',
       b'EXT_SERVER', b'19395_18', b'EXT_SERVER', b'11246_253',
       b'19377_254', b'EXT_SERVER', b'EXT_SERVER', b'18851_139',
       b'EXT_SERVER', b'26913_25', b'EXT_SERVER', b'26914_116',
       b'EXT_SERVER', b'26915_202', b'EXT_SERVER', b'26916_254',
       b'EXT_SERVER', b'10014_199', b'EXT_SERVER', b'10006_27',
       b'EXT_SERVER', b'32955_29', b'EXT_SERVER', b'32955_29',
       b'EXT_SERVER', b'33404_230', b'EXT_SERVER', b'15180_10',
       b'192.168.220.16', b'192.168.220.15', b'192.168.100.5',
       b'192.168.100.5', b'192.168.220.15', b'192.168.100.5',
       b'192.168.100.5', b'192.168.220.9', b'192.168.100.5',
       b'192.168.220.16', b'192.168.220.14', b'192.168.100.5',
       b'192.168.100.5', b'192.168.220.10', b'192.168.210.5',
       b'

## Preprocessing

In [81]:
def _categorical_column(feature_name, vocabulary_list):
    return tf.feature_column.indicator_column(categorical_column=tf.feature_column.categorical_column_with_vocabulary_list(
                                                key=feature_name,
                                                vocabulary_list=vocabulary_list
                                              ))

feature_columns = [
    tf.feature_column.numeric_column(key="src_pt", dtype=tf.int32),
    tf.feature_column.numeric_column(key="dst_pt", dtype=tf.int32),
    tf.feature_column.numeric_column(key="packets", dtype=tf.int64),
    tf.feature_column.numeric_column(key="subnet", dtype=tf.int64),

    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_hash_bucket('src_ip_addr', 10000, tf.string)),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_hash_bucket('dst_ip_addr', 10000, tf.string)),
    _categorical_column('proto',['TCP','UDP']),
    ]

estimator = tf.estimator.LinearClassifier(feature_columns=feature_columns)

labels = csv_df.drop(columns='attackType')
print(csv_df.head())

estimator.train(input_fn=input_fn_train)
#estimator.evaluate(input_fn=input_fn_eval)
#estimator.predict(input_fn=input_fn_predict)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpw625nzzf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb28f9764a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
   Proto    Src IP Addr  Src Pt    Dst IP Addr  Dst Pt  Packets  attackType  \
0  TCP       EXT_SERVER    8082  OPENSTACK_NET   56978     305